In [9]:
# Импортируем необходимые библиотеки
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import pandas as pd
from fastparquet import ParquetFile
from sklearn.metrics import  roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

import pandas as pd
from fastparquet import ParquetFile
import numpy as np

import shap
from utils import feature_drop, remove_highly_correlated_features, get_shap_feature, get_categorical_columns

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

import seaborn as sns
import matplotlib.pyplot as plt
colors = ["#A115FE", "#8F00FF", "#4d0585", "#591878", "#6e1f78", "#7b469c", "#2c0652",
          "#29AB50", "#22753b", "#ECE919", "#7a781d", "#0F91D0", "#0f3f75"]
sns.set_palette(sns.color_palette(colors))

In [6]:
file_path = "Data/train_ai_comp_final_dp.parquet"
pf = ParquetFile(file_path)
df = pf.to_pandas()
df = feature_drop(df)
df.head()

,id,target,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1761,1759,85,105469,191,46,0,6,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1761,1759,120,105610,144,71,0,135,...,0,0,0,0,0,0,0,0,0,0
2,2,0,890,1759,141,105227,191,11,0,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1599,966,30,102441,191,8,0,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1761,1759,85,104006,191,2,0,0,...,0,0,7642,0,0,0,0,0,0,7642


In [10]:
X = df.drop(columns=["target", "id"])
y = df["target"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=666)

In [13]:
classifiers = [
    lgb.LGBMClassifier(n_estimators=50, random_state=42, max_depth=8, learning_rate=0.01),
    XGBClassifier(n_estimators=50, random_state=42, max_depth=8, learning_rate=0.01),
    CatBoostClassifier(iterations=50, learning_rate=0.01, depth=8, loss_function='Logloss', verbose=500)
]
shap_df  = get_shap_feature(X_train, y_train, X_test, classifiers)


c:\Users\Александр\Desktop\УЧЕБА\ХАКАТОНЫ\IT Purple Hack\purple_hack\venv\Lib\site-packages\shap\explainers\_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


0:	learn: 0.6778700	total: 561ms	remaining: 27.5s
49:	learn: 0.2801291	total: 22.6s	remaining: 0us


In [16]:
shap_df.sort_values(by="shap_importance", ascending=False)

,feature,shap_importance
579,feature994,5.021420e-02
200,feature319,3.587303e-02
212,feature341,2.320385e-02
7,feature1004,1.959681e-02
551,feature940,1.559467e-02
...,...,...
176,feature285,3.119086e-07
505,feature889,2.719376e-07
169,feature27,2.502520e-07
569,feature985,1.371030e-07


In [ ]:
feature = shap_df["feature"].values
X_test, X_train= X_test[feature], X_train[feature]
without_drop = shap_df.sort_values(by = "shap_importance", ascending = False)["feature"].head(10)
without_drop.values
columns_to_remove = remove_highly_correlated_features(X_train.drop(columns=without_drop.values), shap_df, threshold=0.9)
X_train, X_test = X_train.drop(columns=columns_to_remove), X_test.drop(columns=columns_to_remove)
category = get_categorical_columns(X_train)

In [ ]:
train = pd.concat([X_train, y_train], axis=1)

In [12]:
task = Task('binary', metric='auc')

tuning_params = {
    'max_tuning_time': 1800,  # Максимальное время тюнинга в секундах
    'max_tuning_iter': 1000,   # Максимальное количество итераций тюнинга
    # Другие параметры тюнинга по необходимости
}

# Настройка AutoML
automl = TabularAutoML(
    task=task,
    timeout=1800 * 3,  # Установите желаемое время в секундах
    cpu_limit=4,
    memory_limit=16,  # Установите лимит памяти в GB
    gpu_ids='0',  # Используйте GPU для ускорения (если доступно)
    reader_params={'cv': 5, 'random_state': 42},
    general_params={'use_algos': [['linear_l2', 'lgb', 'lgb_tuned', 'catboost', 'catboost_tuned', 'xgboost']]},
    tuning_params=tuning_params
)

# Определение ролей
roles = {
    'target': 'target',
    'category': category,  # Пример для категориальных признаков
}

oof_pred = automl.fit_predict(train, roles=roles)


ROC AUC: 0.748963493123685


In [37]:
import pickle

# Сохранение модели
with open('automl_model.pkl', 'wb') as f:
    pickle.dump(automl, f)


with open('automl_model.pkl', 'rb') as f:
    loaded_automl = pickle.load(f)
